In [13]:
import numpy as np


In [100]:
v_sw = 1200
Bx = 1 * 1e-9 #GSM
By = -2 * 1e-9 #GSM
Bz = -50 * 1e-9 #GSM
B = np.sqrt(Bx**2 + By**2 + Bz**2)
mu_0 = np.pi * 4 * 10e-8
l_0 = 6371 * 1000 * 7 #7 R_E

def clock_angle(By, Bz):
    angle = np.rad2deg(np.arctan2(By, Bz))
    np.where(angle < 0, angle + 360, angle)
    return angle

def clock_length(By, Bz):
    return np.sqrt(By**2 + Bz**2)

def cross_polar_cap_pot(Vsw, B, theta):
    #from https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1029/96JA01742
    #and  https://www.sws.bom.gov.au/Category/Solar/Solar%20Conditions/Solar%20Wind%20Clock%20Angle/Solar%20Wind%20Clock%20Angle.php
    return 10e-4 * Vsw**2 + 11.7 * B * (np.sin(np.deg2rad(theta)/2))**3

theta = clock_angle(By, Bz)
length = clock_length(By, Bz)
CPCP = cross_polar_cap_pot(v_sw, length, theta)

epsilon_W = (4 * np.pi / mu_0) * v_sw * B**2 * np.sin(theta / 2)**4 * l_0**2

print(f"ε: {epsilon_W/1e9} GW") #1 watt = 1 Joules

ε: 21.80652136406873 GW


In [13]:
import numpy as np

def create_sequences(target_data, input_data, lookback=30):
    X, y = [], []
    for i in range(len(target_data) - lookback):
        X.append(input_data[i:i+lookback].T)
        y.append(target_data[i+lookback])

    return np.array(X), np.array(y)


target_data = np.arange(100,120,1)
input_data = np.arange(1,20,1)

create_sequences(target_data, input_data, lookback=10)

(array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
        [ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
        [ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        [ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
        [ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]),
 array([110, 111, 112, 113, 114, 115, 116, 117, 118, 119]))